# Lab 02 : Sequence-to-sequence transformers -- solution

### Task   

The goal is to learn to translate an input sequence to an output sequence, which is simply the same input sequence but shifted to the right by one word.  

Example, if the input sequence is "some analysts expect oil prices to remain relatively" then the output sequence is "analysts expect oil prices to remain relatively high".  

We will use an encoder-decoder Transformer to achieve this goal on the PTB dataset. The decoder will start with a token "start_token" assigned to the 10,001-th word in the dictionary. 

Example, if the input sequence is "456 82 948 5892 34 4928 4758 567" then the output sequence is "10001 82 948 5892 34 4928 4758 567" and  the label sequence is "82 948 5892 34 4928 4758 567 745".


In [1]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    path_to_file = '/content/gdrive/My Drive/CS5242_2025_codes/codes/labs_lecture07/lab02_seq2seq/'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

In [3]:
device= torch.device("cuda")
#device= torch.device("cpu")
print(device)

if torch.cuda.is_available():
    print('cuda available with GPU:',torch.cuda.get_device_name(0))

cuda
cuda available with GPU: NVIDIA RTX A5000


### Download Penn Tree Bank

The tensor train_data consists of 20 columns of 46,479 words.<br>
The tensor test_data consists of 20 columns of 4,121 words.

In [4]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'ptb/train_data.pt')
test_data   =  torch.load(data_path+'ptb/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([46479, 20])
torch.Size([4121, 20])


### Some constants associated with the data set

In [5]:
bs = 20
vocab_size = 10000 + 1 # +1 for the start token

### Make an attention net class

In [19]:

def generate_positional_encoding(seq_length, dim):
    assert dim == 2* (dim//2) # check if dim is divisible by 2
    pe = torch.zeros(seq_length, dim)
    position = torch.arange(0, seq_length, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / dim))
    pe[:,0::2] = torch.sin(position * div_term)
    pe[:,1::2] = torch.cos(position * div_term)
    return pe        
    

########### Encoder Transformer Block ###########
class AttentionHead_encoder(nn.Module):
    def __init__(self, d, d_head, dropout):
        super().__init__()
        self.query = nn.Linear(d, d_head, bias=False) # query embedding layer
        self.key = nn.Linear(d, d_head, bias=False) # key embedding layer
        self.value = nn.Linear(d, d_head) # value embedding layer
        self.dropout = nn.Dropout(dropout)
    def forward(self, H): # size(H)=[batch_size, seq_length, d]
        batch_size = H.size(0); batch_len = H.size(1)
        # self-attention encoder
        # Compute a single attention head H = Softmax( QK^T / d^0.5 ) V
        Q = self.query(H) # size=[batch_size, batch_length, d] 
        K = self.key(H) # size=[batch_size, batch_length, d]
        V = self.value(H) # size=[batch_size, batch_length, d]
        attention_score = Q @ K.transpose(2,1) * H.size(2)**-0.5 # QK^T/sqrt(d), (B,L,d) @ (B,d,L) => (B,L,L), size=[batch_size, batch_length, batch_length)
        attention_score = torch.softmax(attention_score, dim=2) # sum weights = 1, size=[batch_size, batch_length, batch_len)
        attention_score = self.dropout(attention_score) # dropout attention scores
        H_HA = attention_score @ V # softmax( QK^T / sqrt(d) ) V, (B,L,L) @ (B,L,d) => (B,L,d), size=[batch_size, batch_length, d)
        return H_HA # return prediction scores for next token
        
class MultipleAttentionHead_encoder(nn.Module):
    def __init__(self, d, num_heads, dropout):
        super().__init__()
        d_head = d // num_heads # dim_head = d // num_heads, usually dimension per head is 64
        assert d == d_head * num_heads # check divisibility
        self.MHA = nn.ModuleList([ AttentionHead_encoder(d, d_head, dropout) for _ in range(num_heads) ])
        self.WO = nn.Linear(d, d) # combination layer
        self.dropout = nn.Dropout(dropout)
    def forward(self, H): # size(H)=[batch_size, seq_length, d]
        batch_size = H.size(0); seq_length = H.size(1)
        H_heads = []
        for HA_layer in self.MHA:
            H_heads.append(HA_layer(H)) # size=[batch_size, seq_length, d_head]
        H_heads = torch.cat(H_heads, dim=2) # size=[batch_size, seq_length, d]            
        H_heads = self.dropout(H_heads) # dropout attention activations
        H = self.WO(H_heads) # size=[batch_size, seq_length, d]
        return H

class TransformerBlock_encoder(nn.Module):
    def __init__(self, d, num_heads, dropout):
        super().__init__()
        self.LN_MHA = nn.LayerNorm(d)
        self.LN_MLP = nn.LayerNorm(d)
        self.MHA = MultipleAttentionHead_encoder(d, num_heads, dropout)
        self.MLP = nn.Sequential(nn.Linear(d,4*d), nn.ReLU(), nn.Dropout(dropout), nn.Linear(4*d,d))   
    def forward(self, H): # size=[batch_size, seq_length, d]
        # Self-attention encoder 
        H = H + self.MHA(self.LN_MHA(H)) # size=[batch_size, seq_length, d], [20, 30, 128]
        # MLP 
        H = H + self.MLP(self.LN_MLP(H)) # size=[batch_size, seq_length, d]
        return H # size=[batch_size, seq_length, d]
    
class Transformer_encoder(nn.Module):
    def __init__(self, d, num_heads, num_blocks, seq_length, dropout):
        super().__init__()
        self.TR_Blocks = nn.ModuleList([ TransformerBlock_encoder(d, num_heads, dropout) for _ in range(num_blocks) ]) 
    def forward(self, batch_seq, pos_enc):
        H = batch_seq.transpose(1,0) # size=[batch_size, seq_length, d]
        batch_size = H.size(0); batch_len = H.size(1)
        # Add positional encoding  
        pos_enc = pos_enc.unsqueeze(dim=0) # size=[1,          seq_length, d]
        H = H + pos_enc                    # size=[batch_size, seq_length, d]
        # Apply transformer blocks 
        for TR_Block in self.TR_Blocks:
            H = TR_Block(H)
        # Output
        H = H.permute(1,0,2)  # size=[batch_length, batch_size, d]
        return H # return prediction scores for next token
########### Encoder Transformer Block ###########
    

########### Decoder Transformer Block ###########
class SelfAttention_AttentionHead_decoder(nn.Module):
    def __init__(self, d, d_head, dropout):
        super().__init__()
        self.query = nn.Linear(d, d_head, bias=False) # query embedding layer
        self.key = nn.Linear(d, d_head, bias=False) # key embedding layer
        self.value = nn.Linear(d, d_head) # value embedding layer
        self.dropout = nn.Dropout(dropout)
    def forward(self, H): # size(H)=[batch_size, seq_length, d]
        batch_size = H.size(0); batch_len = H.size(1)
        # Masked self-attention decoder
        # Compute a single attention head H = Softmax( QK^T / d^0.5 ) V
        Q = self.query(H) # size=[batch_size, batch_length, d] 
        K = self.key(H) # size=[batch_size, batch_length, d]
        V = self.value(H) # size=[batch_size, batch_length, d]
        attention_score = Q @ K.transpose(2,1) * H.size(2)**-0.5 # QK^T/sqrt(d), (B,L,d) @ (B,d,L) => (B,L,L), size=[batch_size, batch_length, batch_length)
        mask = torch.tril(torch.ones(batch_len,batch_len)).long().to(device) # mask to use previous tokens only : { token(<=t) }, size=[batch_len,batch_len]
        attention_score = attention_score.masked_fill(mask==0, value=float('-inf')) # softmax(-inf)=0 prevents using next tokens for prediction, size=(batch_size, batch_len, batch_len)
        attention_score = torch.softmax(attention_score, dim=2) # sum weights = 1, size=[batch_size, batch_length, batch_len)
        attention_score = self.dropout(attention_score) # dropout attention scores
        H_HA = attention_score @ V # softmax( QK^T / sqrt(d) ) V, (B,L,L) @ (B,L,d) => (B,L,d), size=[batch_size, batch_length, d)
        return H_HA # return prediction scores for next token
    
class SelfAttention_MultipleAttentionHead_decoder(nn.Module):
    def __init__(self, d, num_heads, dropout):
        super().__init__()
        d_head = d // num_heads # dim_head = d // num_heads, usually dimension per head is 64
        assert d == d_head * num_heads # check divisibility
        self.MHA = nn.ModuleList([ SelfAttention_AttentionHead_decoder(d, d_head, dropout) for _ in range(num_heads) ])
        self.WO = nn.Linear(d, d) # combination layer
        self.dropout = nn.Dropout(dropout)
    def forward(self, H): # size(H)=[batch_size, seq_length, d]
        batch_size = H.size(0); seq_length = H.size(1)
        H_heads = []
        for HA_layer in self.MHA:
            H_heads.append(HA_layer(H)) # size=[batch_size, seq_length, d_head]
        H_heads = torch.cat(H_heads, dim=2) # size=[batch_size, seq_length, d]            
        H_heads = self.dropout(H_heads) # dropout attention activations
        H = self.WO(H_heads) # size=[batch_size, seq_length, d]
        return H

class CrossAttention_AttentionHead_decoder(nn.Module):
    def __init__(self, d, d_head, dropout):
        super().__init__()
        self.query = nn.Linear(d, d_head, bias=False) # query embedding layer
        self.key = nn.Linear(d, d_head, bias=False) # key embedding layer
        self.value = nn.Linear(d, d_head) # value embedding layer
        self.dropout = nn.Dropout(dropout)
    def forward(self, H, Henc): # size(H)=[batch_size, seq_length, d]
        batch_size = H.size(0); batch_len = H.size(1)
        # Masked cross-attention
        # Compute a single attention head H = Softmax( QK^T / d^0.5 ) V
        Q = self.query(H) # size=[batch_size, batch_length, d]  
        K = self.key(Henc) # size=[batch_size, batch_length, d]
        V = self.value(Henc) # size=[batch_size, batch_length, d]
        attention_score = Q @ K.transpose(2,1) * H.size(2)**-0.5 # QK^T/sqrt(d), (B,L,d) @ (B,d,L) => (B,L,L), size=[batch_size, batch_length, batch_length)
        mask = torch.tril(torch.ones(batch_len,batch_len)).long().to(device) # mask to use previous tokens only : { token(<=t) }, size=[batch_len,batch_len]
        attention_score = attention_score.masked_fill(mask==0, value=float('-inf')) # softmax(-inf)=0 prevents using next tokens for prediction, size=(batch_size, batch_len, batch_len)
        attention_score = torch.softmax(attention_score, dim=2) # sum weights = 1, size=[batch_size, batch_length, batch_len)
        attention_score = self.dropout(attention_score) # dropout attention scores
        H_HA = attention_score @ V # softmax( QK^T / sqrt(d) ) V, (B,L,L) @ (B,L,d) => (B,L,d), size=[batch_size, batch_length, d)
        return H_HA # return prediction scores for next token
        
class CrossAttention_MultipleAttentionHead_decoder(nn.Module):
    def __init__(self, d, num_heads, dropout):
        super().__init__()
        d_head = d // num_heads # dim_head = d // num_heads, usually dimension per head is 64
        assert d == d_head * num_heads # check divisibility
        self.MHA = nn.ModuleList([ CrossAttention_AttentionHead_decoder(d, d_head, dropout) for _ in range(num_heads) ])
        self.WO = nn.Linear(d, d) # combination layer
        self.dropout = nn.Dropout(dropout)
    def forward(self, H, Henc): # size(H)=[batch_size, seq_length, d]
        batch_size = H.size(0); seq_length = H.size(1)
        H_heads = []
        for HA_layer in self.MHA:
            H_heads.append(HA_layer(H, Henc)) # size=[batch_size, seq_length, d_head]
        H_heads = torch.cat(H_heads, dim=2) # size=[batch_size, seq_length, d]            
        H_heads = self.dropout(H_heads) # dropout attention activations
        H = self.WO(H_heads) # size=[batch_size, seq_length, d]
        return H
    
class TransformerBlock_decoder(nn.Module):
    def __init__(self, d, num_heads, dropout):
        super().__init__()
        self.LN_MHA_H = nn.LayerNorm(d)
        self.LN_MHA_Henc = nn.LayerNorm(d)
        self.LN_MLP = nn.LayerNorm(d)
        self.SA_MHA = SelfAttention_MultipleAttentionHead_decoder(d, num_heads, dropout)
        self.CA_MHA = CrossAttention_MultipleAttentionHead_decoder(d, num_heads, dropout)
        self.MLP = nn.Sequential(nn.Linear(d,4*d), nn.ReLU(), nn.Dropout(dropout), nn.Linear(4*d,d))   
    def forward(self, H, Henc): # size=[batch_size, seq_length, d]
        # Masked self-attention decoder
        H = H + self.SA_MHA(self.LN_MHA_H(H)) # size=[batch_size, seq_length, d], [20, 30, 128]
        # Masked cross-attention decoder
        H = H + self.CA_MHA(self.LN_MHA_H(H),self.LN_MHA_Henc(Henc)) # size=[batch_size, seq_length, d], [20, 30, 128]
        # MLP 
        H = H + self.MLP(self.LN_MLP(H)) # size=[batch_size, seq_length, d]
        return H # size=[batch_size, seq_length, d]
           
class Transformer_decoder(nn.Module):
    def __init__(self, d, num_heads, num_blocks, seq_length, dropout):
        super().__init__()
        self.TR_Blocks = nn.ModuleList([ TransformerBlock_decoder(d, num_heads, dropout) for _ in range(num_blocks) ]) 
    def forward(self, g_seq_out, h_enc_seq, pos_enc):
        H = g_seq_out.transpose(1,0) # size=[batch_size, seq_length, d]
        Henc = h_enc_seq.transpose(1,0) # size=[batch_size, seq_length, d]
        batch_size = H.size(0); batch_len = H.size(1)
        # Add positional encoding  
        pos_enc = pos_enc.unsqueeze(dim=0) # size=[1,          seq_length, d]
        H = H + pos_enc                    # size=[batch_size, seq_length, d]
        # Apply transformer blocks 
        for TR_Block in self.TR_Blocks:
            H = TR_Block(H, Henc)
        # Output
        H = H.permute(1,0,2)  # size=[batch_length, batch_size, d]
        return H # return prediction scores for next token
########### Decoder Transformer Block ###########

    
class ANN(nn.Module):
    
    def __init__(self, d, num_heads, num_blocks, seq_length, dropout):
        super(ANN, self).__init__()
        self.encoder = Transformer_encoder(d, num_heads, num_blocks, seq_length, dropout)
        self.decoder = Transformer_decoder(d, num_heads, num_blocks, seq_length, dropout)
    
    def forward(self, g_seq_in , g_seq_out, pos ):
        h_enc_seq = self.encoder( g_seq_in , pos ) # size=[batch_size, seq_length, d], [30, 20, 128]
        h_dec_seq = self.decoder( g_seq_out, h_enc_seq , pos )  # size=[batch_size, seq_length, d], [30, 20, 128]
        return h_dec_seq 
    

class attention_net(nn.Module):

    def __init__(self, d, num_heads, num_blocks, seq_length, dropout):
        super(attention_net, self).__init__()  
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = ANN(d, num_heads, num_blocks, seq_length, dropout)
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

    def forward(self, word_seq_in, word_seq_out, pos ):
        g_seq_in     =   self.layer1( word_seq_in ) # size=(seq_length, bs, hidden_dim), [30, 20, 128]
        g_seq_out     =   self.layer1( word_seq_out ) # size=(seq_length, bs, hidden_dim), [30, 20, 128]
        h_seq     =   self.layer2( g_seq_in , g_seq_out, pos ) # size=(seq_length, bs, hidden_dim), [30, 20, 128] 
        score_seq =   self.layer3( h_seq ) # size=(seq_length, bs, vocab_size)
        return score_seq 


There are 3494417 (3.49 million) parameters in this neural network
minibatch_seq_in torch.Size([30, 20])
minibatch_seq_out torch.Size([30, 20])
minibatch_label torch.Size([30, 20])


### Build the net. Choose the hidden size to be 128 and the number of heads to be 16. 
### How many parameters in total?

In [20]:
hidden_size = 128 
num_heads = 16
num_blocks = 2
seq_length = 100

net = attention_net(hidden_size, num_heads, num_blocks, seq_length, dropout=0.0)
print(net)
utils.display_num_param(net)

attention_net(
  (layer1): Embedding(10001, 128)
  (layer2): ANN(
    (encoder): Transformer_encoder(
      (TR_Blocks): ModuleList(
        (0-1): 2 x TransformerBlock_encoder(
          (LN_MHA): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (LN_MLP): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (MHA): MultipleAttentionHead_encoder(
            (MHA): ModuleList(
              (0-3): 4 x AttentionHead_encoder(
                (query): Linear(in_features=128, out_features=32, bias=False)
                (key): Linear(in_features=128, out_features=32, bias=False)
                (value): Linear(in_features=128, out_features=32, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
            )
            (WO): Linear(in_features=128, out_features=128, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (MLP): Sequential(
            (0): Linear(in_features=128, out_features=512, bias=

### Send the weights of the networks to the GPU

In [21]:
net = net.to(device)

### Choose the loss to be the cross-entropy and the optimizer to be Adam, as well as the hyperparameters: 
* initial learning rate = 0.001
* sequence length = 100

In [22]:
criterion = nn.CrossEntropyLoss()

my_lr = 0.001
seq_length = 100
optimizer = torch.optim.Adam(net.parameters(), lr=my_lr)

pos = generate_positional_encoding(seq_length, hidden_size).to(device) # size=(seq_length, hidden_dim)

### Do 50 passes through the training set
### Observe the train perplexity

In [23]:
start=time.time()
for epoch in range(5):
    
    # set the running quantities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
    doc_len = train_data.size(0)
    for count in range( 0 , doc_len-seq_length ,  seq_length): 
        
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_seq_in = train_data[count:count+seq_length]
        start_token = torch.tensor([vocab_size-1]).repeat(1,bs)
        minibatch_seq_out = torch.cat((start_token, train_data[count+1:count+seq_length]))
        minibatch_label = train_data[count+1:count+seq_length+1]
        
        # send them to the gpu
        minibatch_seq_in = minibatch_seq_in.to(device)
        minibatch_seq_out = minibatch_seq_out.to(device)
        minibatch_label = minibatch_label.to(device)
        
        # forward the minibatch through the net        
        scores = net( minibatch_seq_in, minibatch_seq_out, pos ) # size=(seq_length, bs, vocab_size)

        # reshape the scores and labels to huge batch of size bs*seq_length
        scores = scores.view(  bs*seq_length , vocab_size) # size=(seq_length.bs, vocab_size)
        minibatch_label = minibatch_label.view(  bs*seq_length ) # size=(seq_length.bs, vocab_size)
       
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(scores, minibatch_label)
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        optimizer.step()
        
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    if not epoch%1:
        print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))


epoch= 0 	 time= 14.745864868164062 	 lr= 0.001 	 exp(loss)= 320.7014701865091
epoch= 1 	 time= 28.801368236541748 	 lr= 0.001 	 exp(loss)= 70.56791393642496
epoch= 2 	 time= 42.85297155380249 	 lr= 0.001 	 exp(loss)= 24.058781386409013
epoch= 3 	 time= 56.90769672393799 	 lr= 0.001 	 exp(loss)= 7.4514442855684875
epoch= 4 	 time= 70.96386504173279 	 lr= 0.001 	 exp(loss)= 2.8260608593325607


### Check if the network was successful 

In [24]:
minibatch_seq_in = train_data[count:count+seq_length, 1].unsqueeze(1).to(device)
print('Input sequence:', minibatch_seq_in[:,0])
start_token = torch.tensor([vocab_size-1]).repeat(1,bs).to(device)
minibatch_seq_out = torch.cat((start_token, train_data[count+1:count+seq_length].to(device)))
minibatch_seq_out = minibatch_seq_out[:,1].unsqueeze(1)
minibatch_label = train_data[count+1:count+seq_length+1,1].unsqueeze(1).to(device)
print('\nExpected output sequence:', minibatch_label[:,0])
pos = generate_positional_encoding(seq_length, hidden_size) # size=(seq_length, hidden_dim) 
pos = pos.to(device)
scores = net( minibatch_seq_in, minibatch_seq_out, pos ) # size=(seq_length, bs, vocab_size)
seq = scores.squeeze().argmax(dim=1)
print('\nPredicted output sequence:', seq)
            

Input sequence: tensor([ 275,  225,  853,   88,   64,   65, 6778,  316, 3142, 3939,   24,   26,
         103,   98,   78,   30,  848,   32,   27,   27,   42,   26, 1859, 1467,
         189,   99,  119, 6779, 6780,   93,   78,  250,  251, 1190,  992,   24,
        1187,  113,  251, 1103,   24,   26,  448,   27,   27,   42,   32, 6780,
         227,   69,  482,  108,   26, 6781,   48,   26,   26,   24,   26,  227,
        4190,  972, 1467,  548,  108,   27, 2876,   98,   78, 4044,  109, 1485,
          42,  458, 3396, 1514,  128,  830,  169,  128, 3659, 3017, 1842,   64,
         848,   32, 3186,  546,  108,   32, 6780,   24, 4755,  230, 6161, 3155,
          99,   98,   78,  980], device='cuda:0')

Expected output sequence: tensor([ 225,  853,   88,   64,   65, 6778,  316, 3142, 3939,   24,   26,  103,
          98,   78,   30,  848,   32,   27,   27,   42,   26, 1859, 1467,  189,
          99,  119, 6779, 6780,   93,   78,  250,  251, 1190,  992,   24, 1187,
         113,  251, 1103,  